In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from fastFM import als, mcmc, sgd
# from pyfm import pylibfm

import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
catboost_columns = ['spkt_welch_density__coeff_3',
 'spkt_welch_densitycoeff_2',
 'abs_q25_5',
 'abs_q75_6',
 'q05_roll_std_1000',
 'abs_q75_7',
 'abs_q95_2',
 'q05_5',
 'abs_q75_2',
 '5000skewness_max_',
 'fft_coefficientcoeff_80__attr_"imag"',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 '5000kurtosis_mean_',
 "number_peaks{'n': 1}",
 '5000smoothness_entropy_',
 'ave10_7',
 'q75_roll_std_1000',
 'FFT_Mag_25q0',
 'fft_coefficientcoeff_20__attr_"abs"']
lgbm_columns = ['q25_roll_std_100',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_6',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 'median__roll_std',
 'abs_q01_5',
 '5000smoothness_quantile05',
 '5000smoothness_std_',
 'abs_q95_3',
 'FFT_Mag_75q0',
 '5000median_std_',
 'spkt_welch_density__coeff_17']
xgbm_columns = ['q25_roll_std_100',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 "number_peaks{'n': 1}",
 '5000peak_peak_amp_max_',
 'abs_q95_3',
 'spkt_welch_density__coeff_89',
 'abs_q05_2']
randomforest_randomforest = ['abs_q25_5', 'abs_q01_4', 'q25_roll_std_100']
extratrees_columns = ['q05_2',
 "number_peaks{'n': 1}",
 'abs_q01_6',
 'abs_q95_2',
 '5000smoothness_quantile25',
 '5000std_median_',
 '5000smoothness_median_',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'abs_q75_6',
 "number_peaks{'n': 3}",
 'q75_roll_std_10',
 "number_peaks{'n': 10}",
 '5000min_quantile75',
 '5000smoothness_quantile05',
 "number_peaks{'n': 5}",
 'abs_q01_2',
 '5000smoothness_mean_',
 'min_roll_std_100',
 'abs_q05_2',
 'q01_roll_std_1000']
gradientboosting_columns = ['q05_5',
 'kurt_1',
 'abs_q75_6',
 'abs_q75_7',
 'spkt_welch_density__coeff_28',
 'spkt_welch_density__coeff_99',
 'fft_coefficientcoeff_6__attr_"abs"',
 '5000smoothness_quantile05',
 'q25_roll_std_100',
 'spkt_welch_densitycoeff_2',
 'abs_max_1',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'abs_q25_5',
 'abs_q01_7',
 'q05_8',
 'Hilbert_mean_6',
 'abs_q95_2',
 '5000skewness_max_',
 '5000kurtosis_mean_',
 'spkt_welch_density__coeff_3']

In [9]:
all_columns = catboost_columns+lgbm_columns+xgbm_columns+randomforest_randomforest+extratrees_columns+gradientboosting_columns
unique_columns = list(set(all_columns))
common_columns = []
common_columns50 = []
common_columns75 = []
common_columns95 = []
N_columns = 6
count_values = [all_columns.count(col) for col in unique_columns]
for col in unique_columns:
    if all_columns.count(col)==N_columns:
        common_columns.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .5):
        common_columns50.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .75):
        common_columns75.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .95):
        common_columns95.append(col)
print('unique_columns ',len(unique_columns))
print('common_columns50 ',len(common_columns50))
print('common_columns75 ',len(common_columns75))
print('common_columns95 ',len(common_columns95))
print('common_columns ',len(common_columns))

unique_columns  55
common_columns50  55
common_columns75  15
common_columns95  7
common_columns  0


In [10]:
# mytrial = []
db = DFDB('../trial/fm.pkl', auto_commit=False)

In [13]:
param = {'algorithm': {'cls': 'als.FMRegression',
  'fit': {},
  'init': {'n_iter': 10,
   'init_stdev': 0.00030963137584220923,
   'rank': 2,
   'random_state': 42,
   'l2_reg_w': 0.1,
   'l2_reg_V': 0.1,
   'l2_reg': 0}},
 'columns': common_columns95,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init':{}}}


mytrial = []
selected_columns = EP.revert_rfe(df_train, param, common_columns50, df_test, mytrial, start_columns=common_columns95, limit=20, remark='start from top1 column')
print(len(selected_columns))
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

20


In [17]:
selected_columns=['abs_q75_7',
 'spkt_welch_densitycoeff_2',
 'abs_q75_6',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'spkt_welch_density__coeff_3',
 'q25_roll_std_100',
 'abs_q25_5',
 "number_peaks{'n': 1}",
 'ave10_7',
 '5000smoothness_quantile05',
 'abs_max_1',
 'FFT_Mag_75q0',
 'median__roll_std',
 '5000kurtosis_mean_',
 'abs_q01_4',
 'kurt_1',
 'q05_roll_std_1000',
 '5000smoothness_quantile25',
 '5000std_median_',
 'abs_q95_3']

In [18]:
db.commit()

In [20]:
mytrial = []
#  tune hypterparameters
def objective(trial):

    n_iter = trial.suggest_int('n_iter', 10, 100)
    init_stdev = trial.suggest_uniform('init_stdev', 0.00001, .01)
    rank = trial.suggest_int('rank', 2, 16)
    random_state = trial.suggest_int('random_state', 0, 9999)
        
    args={'algorithm': {'cls': 'als.FMRegression',
      'fit': {},
      'init': {'n_iter': n_iter,
       'init_stdev': init_stdev,
       'rank': rank,
       'random_state': random_state,}},
     'columns': selected_columns,
     'kfold': {'n_splits': 3,
      'random_state': 1985,
      'shuffle': True,
      'type': 'group'},
     'scaler': {'cls': 'StandardScaler', 'init':{}}}
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune group3')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-26 01:17:20,583] Finished a trial resulted in value: 0.04662901881903619. Current best value is 0.04662901881903619 with parameters: {'n_iter': 45, 'init_stdev': 0.009538282686827316, 'rank': 2, 'random_state': 7656}.
[I 2019-05-26 01:17:30,386] Finished a trial resulted in value: 0.0844894641673616. Current best value is 0.04662901881903619 with parameters: {'n_iter': 45, 'init_stdev': 0.009538282686827316, 'rank': 2, 'random_state': 7656}.
[I 2019-05-26 01:17:39,677] Finished a trial resulted in value: 0.05988422215942953. Current best value is 0.04662901881903619 with parameters: {'n_iter': 45, 'init_stdev': 0.009538282686827316, 'rank': 2, 'random_state': 7656}.
[I 2019-05-26 01:17:51,680] Finished a trial resulted in value: 0.0862039746797625. Current best value is 0.04662901881903619 with parameters: {'n_iter': 45, 'init_stdev': 0.009538282686827316, 'rank': 2, 'random_state': 7656}.
[I 2019-05-26 01:17:57,456] Finished a trial resulted in value: 0.05510623965717084. C

[I 2019-05-26 01:21:16,616] Finished a trial resulted in value: 0.059969935059648476. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:21:20,907] Finished a trial resulted in value: 0.048630130331478325. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:21:31,160] Finished a trial resulted in value: 0.08209736981571261. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:21:33,053] Finished a trial resulted in value: 0.05335582114368856. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:21:38,241] Finished a trial resulted in value: 0.0568511376790082

[I 2019-05-26 01:24:50,610] Finished a trial resulted in value: 0.050359676767438646. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:24:52,376] Finished a trial resulted in value: 0.046054037027527334. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:24:55,435] Finished a trial resulted in value: 0.07192484921264658. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:24:56,546] Finished a trial resulted in value: 0.04566146961339898. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:25:02,305] Finished a trial resulted in value: 0.0582754728876981

[I 2019-05-26 01:26:58,901] Finished a trial resulted in value: 0.04739188461819825. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:27:10,909] Finished a trial resulted in value: 0.08458854098904746. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:27:13,927] Finished a trial resulted in value: 0.0614522892168149. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:27:32,075] Finished a trial resulted in value: 0.10162101040772704. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:27:33,424] Finished a trial resulted in value: 0.053903547664277636.

[I 2019-05-26 01:30:08,565] Finished a trial resulted in value: 0.04400806880011681. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:30:09,888] Finished a trial resulted in value: 0.04711054268823684. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:30:15,218] Finished a trial resulted in value: 0.06419255815340304. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:30:17,693] Finished a trial resulted in value: 0.05358427804991516. Current best value is 0.03904580656126248 with parameters: {'n_iter': 10, 'init_stdev': 0.0054276840247078845, 'rank': 2, 'random_state': 189}.
[I 2019-05-26 01:30:21,385] Finished a trial resulted in value: 0.060568498061312016

[I 2019-05-26 01:31:31,309] Finished a trial resulted in value: 0.059870304423176444. Current best value is 0.03710678647790195 with parameters: {'n_iter': 13, 'init_stdev': 0.0061630718526306475, 'rank': 2, 'random_state': 2568}.
[I 2019-05-26 01:31:33,611] Finished a trial resulted in value: 0.050352610273297425. Current best value is 0.03710678647790195 with parameters: {'n_iter': 13, 'init_stdev': 0.0061630718526306475, 'rank': 2, 'random_state': 2568}.
[I 2019-05-26 01:31:37,530] Finished a trial resulted in value: 0.05045494478710502. Current best value is 0.03710678647790195 with parameters: {'n_iter': 13, 'init_stdev': 0.0061630718526306475, 'rank': 2, 'random_state': 2568}.
[I 2019-05-26 01:31:43,079] Finished a trial resulted in value: 0.05523208549097079. Current best value is 0.03710678647790195 with parameters: {'n_iter': 13, 'init_stdev': 0.0061630718526306475, 'rank': 2, 'random_state': 2568}.
[I 2019-05-26 01:31:45,681] Finished a trial resulted in value: 0.050633746714

In [24]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [25]:
df_trial[(df_trial['remark']=='tune group3')&(df_trial['mae_diff']<.1)].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].head()

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
686,2019-05-26 01:28:33.386373,20,2.051797,0.003775,2.077350,0.015724,0.025554,tune group3
486,2019-05-26 01:28:33.386373,20,2.051797,0.003775,2.077350,0.015724,0.025554,tune group3
559,2019-05-26 01:32:39.863898,20,2.046807,0.003556,2.078483,0.015152,0.031675,tune group3
759,2019-05-26 01:32:39.863898,20,2.046807,0.003556,2.078483,0.015152,0.031675,tune group3
745,2019-05-26 01:31:43.061524,20,2.052291,0.004404,2.078860,0.015118,0.026568,tune group3


In [26]:
mytrial = []
#  tune hypterparameters
def objective(trial):

    n_iter = trial.suggest_int('n_iter', 10, 100)
    init_stdev = trial.suggest_uniform('init_stdev', 0.00001, .01)
    rank = trial.suggest_int('rank', 2, 16)
    random_state = trial.suggest_int('random_state', 0, 9999)
        
    args={'algorithm': {'cls': 'als.FMRegression',
      'fit': {},
      'init': {'n_iter': n_iter,
       'init_stdev': init_stdev,
       'rank': rank,
       'random_state': random_state,}},
     'columns': selected_columns,
     'kfold': {'n_splits': 3,
      'random_state': 1985,
      'shuffle': True,
      'type': 'stratified'},
     'scaler': {'cls': 'StandardScaler', 'init':{}}}
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune stratified3')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-26 01:34:49,724] Finished a trial resulted in value: 0.007506201840316073. Current best value is 0.007506201840316073 with parameters: {'n_iter': 23, 'init_stdev': 0.0021415421635656, 'rank': 7, 'random_state': 2456}.
[I 2019-05-26 01:35:12,394] Finished a trial resulted in value: 0.016280211013993323. Current best value is 0.007506201840316073 with parameters: {'n_iter': 23, 'init_stdev': 0.0021415421635656, 'rank': 7, 'random_state': 2456}.
[I 2019-05-26 01:35:18,867] Finished a trial resulted in value: 0.01121723165948044. Current best value is 0.007506201840316073 with parameters: {'n_iter': 23, 'init_stdev': 0.0021415421635656, 'rank': 7, 'random_state': 2456}.
[I 2019-05-26 01:35:21,114] Finished a trial resulted in value: 0.011676355042944864. Current best value is 0.007506201840316073 with parameters: {'n_iter': 23, 'init_stdev': 0.0021415421635656, 'rank': 7, 'random_state': 2456}.
[I 2019-05-26 01:35:28,523] Finished a trial resulted in value: 0.008394208274894555.

[I 2019-05-26 01:38:35,831] Finished a trial resulted in value: 0.008195529560585339. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:38:39,497] Finished a trial resulted in value: 0.008224352797663861. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:38:42,205] Finished a trial resulted in value: 0.008575686796423565. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:38:49,377] Finished a trial resulted in value: 0.013049677452149896. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:38:54,625] Finished a trial resulted in value: 0.0103630915

[I 2019-05-26 01:41:50,320] Finished a trial resulted in value: 0.0088616868144813. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:41:53,778] Finished a trial resulted in value: 0.007521303441762324. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:41:57,086] Finished a trial resulted in value: 0.011969954416199323. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:42:05,576] Finished a trial resulted in value: 0.01217042814048919. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:42:08,669] Finished a trial resulted in value: 0.0098452093046

[I 2019-05-26 01:44:57,570] Finished a trial resulted in value: 0.01248548494641704. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:45:01,342] Finished a trial resulted in value: 0.010397867101829364. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:45:03,365] Finished a trial resulted in value: 0.00787225002346233. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:45:06,511] Finished a trial resulted in value: 0.010085947225272458. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:45:07,438] Finished a trial resulted in value: 0.007489096574

[I 2019-05-26 01:47:47,223] Finished a trial resulted in value: 0.012735851651273566. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:47:52,685] Finished a trial resulted in value: 0.009831400431316454. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:48:00,673] Finished a trial resulted in value: 0.01470114089574135. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:48:24,971] Finished a trial resulted in value: 0.017032773024528252. Current best value is 0.005934186443309905 with parameters: {'n_iter': 17, 'init_stdev': 0.0018351623512978148, 'rank': 3, 'random_state': 708}.
[I 2019-05-26 01:48:29,724] Finished a trial resulted in value: 0.00809231949

[I 2019-05-26 01:49:55,116] Finished a trial resulted in value: 0.012793312327122724. Current best value is 0.005104692238407495 with parameters: {'n_iter': 11, 'init_stdev': 0.0012279813119999206, 'rank': 5, 'random_state': 8065}.
[I 2019-05-26 01:49:56,777] Finished a trial resulted in value: 0.00804182325813722. Current best value is 0.005104692238407495 with parameters: {'n_iter': 11, 'init_stdev': 0.0012279813119999206, 'rank': 5, 'random_state': 8065}.
[I 2019-05-26 01:49:59,650] Finished a trial resulted in value: 0.011537490966771017. Current best value is 0.005104692238407495 with parameters: {'n_iter': 11, 'init_stdev': 0.0012279813119999206, 'rank': 5, 'random_state': 8065}.
[I 2019-05-26 01:50:03,362] Finished a trial resulted in value: 0.012812154524252873. Current best value is 0.005104692238407495 with parameters: {'n_iter': 11, 'init_stdev': 0.0012279813119999206, 'rank': 5, 'random_state': 8065}.
[I 2019-05-26 01:50:14,332] Finished a trial resulted in value: 0.0118935

In [27]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [28]:
df_trial[(df_trial['remark']=='tune stratified3')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].head()

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
768,2019-05-26 01:35:12.392853,20,2.041783,0.000002,2.049726,0.000064,0.007943,tune stratified3
808,2019-05-26 01:39:24.783206,20,2.042842,0.000009,2.050372,0.000061,0.007529,tune stratified3
910,2019-05-26 01:48:24.955989,20,2.043256,0.000001,2.051559,0.000068,0.008302,tune stratified3
906,2019-05-26 01:47:42.190634,20,2.048429,0.000018,2.053360,0.000062,0.004932,tune stratified3
864,2019-05-26 01:44:09.071331,20,2.046135,0.000004,2.054485,0.000070,0.008350,tune stratified3


In [29]:
db.commit()